In [1]:
import torch
import sklearn
import gensim
from gensim import downloader
import numpy as np
from sklearn.metrics import f1_score

In [2]:
!pip install gensim


     |████████████████████████████████| 24.1 MB 8.0 MB/s eta 0:00:01     |███████████████████████████▌    | 20.7 MB 6.3 MB/s eta 0:00:01
     |████████████████████████████████| 60 kB 6.3 MB/s  eta 0:00:01


In [45]:
import string

"""[('paulwalk', 0), ('it', 0), ('s', 0), ('the', 0), ('view', 0), ('from', 0), ('where', 0), ('i', 0), ('m', 0), ('living', 0), ('for', 0), ('two', 0), ('weeks', 0), ('empire', 1), ('state', 1), ('building', 1), ('esb', 1), ('pretty', 0), ('bad', 0), ('storm', 0), ('here', 0), ('last', 0), ('evening', 0)]"""
def extract_sentences_with_tags(file_path, untagged):
    sentences = []
    current_sentence = []
    
#     # Define a translation table to remove punctuation
#     punctuation_marks = {'.', ',', '=', '...', '!', '?', ':', ';', '&lt;', '&gt;', '-', '(', ')', '[', ']', '{', '}',
#                          '"', "'"}
    
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            line = line.strip()
            if line:
                # Split the line into word and label
                if untagged:
                    parts = [line, '_']
                else:
                    parts = line.split('\t')
                    if len(parts) != 2:
                        continue
                    

                word, tag = parts
                binary_tag = 0 if tag == 'O' else 1
                word = word.lower()  # Convert word to lowercase
                
#                 if word in punctuation_marks:
#                     continue  # Skip words starting with @ or #, and punctuation marks
                    
                if not any(char.isalpha() for char in word):
                    continue
                
                elif current_sentence and (word.startswith("'") or word == "n't"):  # Handle contractions and negations
                    # Combine with previous word and keep the most specific tag (non-'O' if present)
                    prev_word, prev_tag = current_sentence[-1]
                    current_sentence[-1] = (prev_word + word, prev_tag if prev_tag != 0 else binary_tag)
                else:
                    current_sentence.append((word, binary_tag))
#                 if untagged:
#                     current_sentence.append(word)
#                 else:
#                     try:
#                         label = parts[1]
#                     except: continue
                    # Convert label to binary (0 for 'O', 1 otherwise)
#                 i
    
#                     binary_label = 0 if tag == 'O' else 1
#                     current_sentence.append((word, binary_label))
            else:
                if current_sentence:  # Only add non-empty sentences
                    sentences.append(current_sentence)
                    current_sentence = []
                    
        # Add the last sentence if file doesn't end with a blank line
        if current_sentence:
            sentences.append(current_sentence)

    return sentences



In [49]:

def sentence_to_embedding(sentence, model, embedding_dim=200):
    # Initialize an empty list to hold the embeddings
    embedding_matrix = []
    
    for word in sentence:
        if word in model.key_to_index:
            embedding_matrix.append(model[word])
        else:
            if len(embedding_matrix) > 3:
                former_vectors = [vec for vec in embedding_matrix[-3:]]
                vec = np.mean(former_vectors, axis=0)
            else:
                former_vectors = [vec for vec in embedding_matrix]
                for i in range(2 - len(embedding_matrix)):
                    former_vectors.append(np.random.rand(model.vector_size))
                vec = np.mean(former_vectors, axis=0)
            embedding_matrix.append(vec)
    
    return embedding_matrix

# def sentence_to_embedding(sentence, model, embedding_dim=300, window_size=2):
#     # Initialize an empty list to hold the embeddings
#     embedding_matrix = []
    
#     for i, word in enumerate(sentence):
#         if word in model.key_to_index:
#             embedding_matrix.append(model[word])
#         else:
#             # If the word is not in the vocabulary, try to use the context
#             context_embeddings = []
#             # Look before the current word
#             for j in range(max(0, i-window_size), i):
#                 if sentence[j] in model.key_to_index:
#                     context_embeddings.append(model[sentence[j]])
#             # Look after the current word
#             for j in range(i+1, min(len(sentence), i+window_size+1)):
#                 if sentence[j] in model.key_to_index:
#                     context_embeddings.append(model[sentence[j]])
            
#             # If we found any context words, average their embeddings
#             if context_embeddings:
#                 avg_embedding = np.mean(context_embeddings, axis=0)
#                 embedding_matrix.append(avg_embedding)
#             else:
#                 # If no context words found or all context words are also OOV, use a zero vector
#                 embedding_matrix.append(np.zeros(embedding_dim))
    
#     return np.array(embedding_matrix)


In [36]:
def split_sentences(sen_with_tags):
    X = []
    y = []
    for sentence in sen_with_tags:
        X.append([word for word, tag in sentence])
        y.append([tag for word, tag in sentence])
    return X, y
        

In [50]:
def pad_embeddings(embeddings, max_len, embedding_dim=200):
    padded_embeddings = []
    for sentence in embeddings:
        # Calculate how many zeros need to be added
        padding_size = max_len - len(sentence)
        if padding_size > 0:
            # Pad the sentence with zero vectors at the end
            sentence = np.vstack((sentence, np.zeros((padding_size, embedding_dim))))
        padded_embeddings.append(sentence)
    return np.array(padded_embeddings)

In [27]:
def pad_labels(labels, max_len, padding_label):
    padded_labels = []
    for label_seq in labels:
        # Calculate how many padding labels need to be added
        padding_size = max_len - len(label_seq)
        if padding_size > 0:
            # Extend the label sequence with the padding label
            label_seq.extend([padding_label] * padding_size)
        padded_labels.append(label_seq)
    return padded_labels

# Assuming `labels` is a list of lists where each sublist contains labels for a sentence
# and `max_len` is the maximum sentence length determined previously
# padded_labels = pad_labels(labels, max_len)


In [47]:
from gensim.models import Word2Vec
import torch
from torch.utils.data import Dataset
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from gensim import downloader

# we want word2vec representation
# WORD_2_VEC_PATH = 'word2vec-google-news-300'
# word2vec_model = downloader.load(WORD_2_VEC_PATH)
GLOVE_PATH = 'glove-twitter-200'
# word2vec_model = downloader.load(GLOVE_PATH)

[=-------------------------------------------------] 3.1% 23.4/758.5MB downloaded

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



[====----------------------------------------------] 8.4% 63.4/758.5MB downloaded

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



[======--------------------------------------------] 13.9% 105.7/758.5MB downloaded

...

[============================================------] 89.7% 680.7/758.5MB downloaded

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



[================================================--] 96.7% 733.3/758.5MB downloaded

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



[==================================================] 100.0% 758.5/758.5MB downloaded


In [28]:
from torch import nn


class FF_NN(nn.Module):

    def __init__(self, vec_dim, num_classes, hidden_dim=100):
        super(FF_NN, self).__init__()
        self.ignore_index = -100
        self.first_layer = nn.Linear(vec_dim, hidden_dim)
        self.second_layer = nn.Linear(hidden_dim, num_classes)
        self.activation = nn.ReLU()
        self.loss = nn.CrossEntropyLoss()
        self.num_classes = num_classes


    def forward(self, input_ids, labels=None):
        x = self.first_layer(input_ids)
        x = self.activation(x)
        x = self.second_layer(x)  # Logits: [batch_size, sequence_length, num_classes]

        if labels is not None:
            # Flatten logits and labels to align with CrossEntropyLoss expectations
            logits_flat = x.view(-1, x.size(-1))  # [batch_size * sequence_length, num_classes]
            labels_flat = labels.view(-1)  # [batch_size * sequence_length]
            loss = self.loss(logits_flat, labels_flat)
            return logits_flat, loss

        # If no labels provided, return logits in their original shape for inference
        return x, None

#     def forward(self, input_ids, labels=None):
#         x = self.first_layer(input_ids)
#         x = self.activation(x)
#         x = self.second_layer(x)

#         # Check if labels were provided and are not already a tensor
#         if labels is not None:
#             if not isinstance(labels, torch.Tensor):
#                 # Convert labels to a tensor if they are not already
#                 # Ensure labels are on the same device as the model
#                 labels = torch.tensor(labels, dtype=torch.long, device=x.device)

#             # Compute the loss
#             loss = self.loss(x, labels)
#             return x, loss

#         # If labels are not provided, just return the logits
#         return x, None


In [29]:
class NERDataSet(Dataset):

    def __init__(self, file_path, vector_type, untagged):
        self.file_path = file_path
        data = extract_sentences_with_tags(file_path, untagged)
        
        # Assuming `sentences` is your list of sentences (as lists of words) and `labels` is your list of labels
        sentences, labels = split_sentences(data)

        self.sentences = sentences
        self.labels = labels
        
        all_labels = [label for sublist in self.labels for label in sublist]
        unique_labels = sorted(set(all_labels))
        self.tags_to_idx = {tag: idx for idx, tag in enumerate(unique_labels)}
        self.idx_to_tag = {idx: tag for tag, idx in self.tags_to_idx.items()}
        
        self.labels_idx = [[self.tags_to_idx[label] for label in label_seq] for label_seq in labels]

        
        self.vector_type = vector_type
        
        
        if vector_type == 'w2v':
            model = downloader.load(WORD_2_VEC_PATH)
        elif vector_type == 'glove':
            model = downloader.load(GLOVE_PATH)
        else:
            raise KeyError(f"{vector_type} is not a supported vector type")
            
        # Process sentences to embeddings
        self.tokenized_sen = [sentence_to_embedding(sen, model, embedding_dim=300) for sen in self.sentences]
        self.max_len = max(len(sen) for sen in self.tokenized_sen)

        # Apply padding
        self.tokenized_sen = pad_embeddings(self.tokenized_sen, self.max_len)
        self.ignore_index = -100
        self.labels_idx = pad_labels(self.labels_idx, self.max_len, self.ignore_index)
        
        self.vector_dim = self.tokenized_sen.shape[-1]

        
        
    def __getitem__(self, item):
        cur_sen = self.tokenized_sen[item]
        cur_labels = self.labels_idx[item]

        cur_sen = torch.FloatTensor(cur_sen)
        cur_labels = torch.LongTensor(cur_labels)

        return {"input_ids": cur_sen, "labels": cur_labels}

    def __len__(self):
        return len(self.tokenized_sen)

In [40]:
from sklearn.metrics import accuracy_score, f1_score
from torch.utils.data import DataLoader
from torch.optim import Adam

# def train(model, data_sets, optimizer, num_epochs: int, batch_size=16):
#     device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#     model.to(device)
#     criterion = torch.nn.CrossEntropyLoss(ignore_index=model.ignore_index)

#     data_loaders = {
#         "train": DataLoader(data_sets["train"], batch_size=batch_size, shuffle=True),
#         "test": DataLoader(data_sets["test"], batch_size=batch_size, shuffle=False)
#     }

#     best_acc = 0.0

#     for epoch in range(num_epochs):
#         print(f'Epoch {epoch + 1}/{num_epochs}')
#         print('-' * 10)

#         for phase in ['train', 'test']:
#             model.train() if phase == 'train' else model.eval()

#             running_loss = 0.0
#             true_labels = []
#             predictions = []

#             for batch in data_loaders[phase]:
#                 inputs = batch["input_ids"].to(device)
#                 labels = batch["labels"].to(device) if phase == 'train' else None
                
#                 optimizer.zero_grad()

#                 with torch.set_grad_enabled(phase == 'train'):
#                     # Directly use the model's output for logits and loss
#                     outputs, loss = model(inputs, labels) if labels is not None else model(inputs)

#                     if loss is None and labels is not None:
#                         # If the model doesn't calculate loss (e.g., during inference or specific conditions),
#                         # we need to calculate it manually for the training phase.
#                         outputs_flat = outputs.view(-1, model.num_classes)
#                         labels_flat = labels.view(-1)
#                         loss = criterion(outputs_flat, labels_flat)

#                     if phase == 'train' and loss is not None:
#                         loss.backward()
#                         optimizer.step()

#                     # For accuracy calculation, ensure logits are in the correct shape
#                     # and only calculate for non-None labels
#                     if labels is not None:
#                         logits = outputs if loss is None else outputs.view(-1, model.num_classes)
#                         _, preds = torch.max(logits, dim=1)
#                         labels_flat = labels.view(-1)
#                         active = labels_flat != model.ignore_index
#                         true_labels.extend(labels_flat[active].cpu().numpy())
#                         predictions.extend(preds[active].cpu().numpy())

#                     running_loss += loss.item() * inputs.size(0) if loss is not None else 0

#             epoch_loss = running_loss / len(data_loaders[phase].dataset) if loss is not None else 0
#             epoch_acc = accuracy_score(true_labels, predictions) if labels is not None else 0

#             print(f'{phase.title()} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')
            
#             if phase == 'test' and epoch_acc > best_acc:
#                 best_acc = epoch_acc
#                 torch.save(model.state_dict(), 'best_model.pth')

#         print()

#     print(f'Best Test Accuracy: {best_acc:.4f}')

import torch
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score

# def train(model, data_sets, optimizer, num_epochs: int, batch_size=16):
#     device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#     criterion = torch.nn.CrossEntropyLoss(ignore_index=model.ignore_index)
#     model.to(device)

#     data_loaders = {
#         "train": DataLoader(data_sets["train"], batch_size=batch_size, shuffle=True),
#         "test": DataLoader(data_sets["test"], batch_size=batch_size, shuffle=False)
#     }

#     best_acc = 0.0

#     for epoch in range(num_epochs):
#         print(f'Epoch {epoch + 1}/{num_epochs}')
#         print('-' * 10)

#         for phase in ['train', 'test']:
#             model.train() if phase == 'train' else model.eval()

#             running_loss = 0.0
#             total_samples = 0
#             correct_predictions = 0

#             for batch in data_loaders[phase]:
#                 inputs = batch["input_ids"].to(device)
#                 labels = batch["labels"].to(device)

#                 optimizer.zero_grad()

#                 with torch.set_grad_enabled(phase == 'train'):
#                     outputs, _ = model(inputs)
#                     loss = None

#                     if phase == 'train':
#                         # Flatten logits and labels to compute the loss for training
#                         loss = criterion(outputs.view(-1, model.num_classes), labels.view(-1))
#                         loss.backward()
#                         optimizer.step()
#                     elif phase == 'test':
#                         # No need to flatten for loss computation in test phase
#                         loss = criterion(outputs.view(-1, model.num_classes), labels.view(-1))

#                     # Calculate accuracy
#                     preds = torch.argmax(outputs, dim=2)  # Predictions across num_classes dimension
#                     preds = preds.view(-1)  # Flatten predictions
#                     labels_flat = labels.view(-1)  # Flatten true labels
#                     correct = (preds == labels_flat) & (labels_flat != model.ignore_index)  # Exclude ignore_index in accuracy calculation
#                     correct_predictions += correct.sum().item()
#                     total_samples += correct.size(0)

#                     if loss is not None:
#                         running_loss += loss.item() * inputs.size(0)

#             epoch_loss = running_loss / len(data_loaders[phase].dataset)
#             epoch_acc = correct_predictions / total_samples

#             print(f'{phase.title()} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

#             if phase == 'test' and epoch_acc > best_acc:
#                 best_acc = epoch_acc
#                 torch.save(model.state_dict(), 'best_model.pth')

#         print()

#     print(f'Best Test Accuracy: {best_acc:.4f}')


from sklearn.metrics import f1_score
import torch
from torch.utils.data import DataLoader

def train(model, data_sets, optimizer, num_epochs: int, batch_size=2):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    criterion = torch.nn.CrossEntropyLoss(ignore_index=model.ignore_index)
    model.to(device)

    data_loaders = {
        "train": DataLoader(data_sets["train"], batch_size=batch_size, shuffle=True),
        "test": DataLoader(data_sets["test"], batch_size=batch_size, shuffle=False)
    }

    best_f1 = 0.0

    for epoch in range(num_epochs):
        print(f'Epoch {epoch + 1}/{num_epochs}')
        print('-' * 10)

        for phase in ['train', 'test']:
            model.train() if phase == 'train' else model.eval()

            running_loss = 0.0
            all_preds = []
            all_labels = []

            for batch in data_loaders[phase]:
                inputs = batch["input_ids"].to(device)
                labels = batch["labels"].to(device)

                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'train'):
                    outputs, _ = model(inputs)
                    loss = criterion(outputs.view(-1, model.num_classes), labels.view(-1))

                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                    # Store predictions and true labels for F1 score calculation
                    preds = torch.argmax(outputs, dim=2).view(-1)  # Flatten predictions
                    labels_flat = labels.view(-1)  # Flatten true labels
                    valid_indices = labels_flat != model.ignore_index  # Exclude ignore_index

                    valid_preds = preds[valid_indices].cpu().numpy()
                    valid_labels = labels_flat[valid_indices].cpu().numpy()

                    all_preds.extend(valid_preds)
                    all_labels.extend(valid_labels)

                    running_loss += loss.item() * inputs.size(0)

            epoch_loss = running_loss / len(data_loaders[phase].dataset)
            epoch_f1 = f1_score(all_labels, all_preds)

            print(f'{phase.title()} Loss: {epoch_loss:.4f} F1: {epoch_f1:.4f}')

            if phase == 'test' and epoch_f1 > best_f1:
                best_f1 = epoch_f1
                torch.save(model.state_dict(), 'best_model.pth')

        print()

    print(f'Best Test F1: {best_f1:.4f}')



In [51]:
import torch
from torch.utils.data import DataLoader, TensorDataset
from torch.optim import Adam

train_path = '/Users/ranykhirbawi/Desktop/data 2/train.tagged'
test_path = '/Users/ranykhirbawi/Desktop/data 2/dev.tagged'

train_ds = NERDataSet(train_path, vector_type='glove', untagged=False)
print('created train')
test_ds = NERDataSet(test_path, vector_type='glove', untagged=False)


datasets = {"train": train_ds, "test": test_ds}
FF_model = FF_NN(num_classes=2, vec_dim=train_ds.vector_dim)
optimizer = Adam(params=FF_model.parameters())
train(model=FF_model, data_sets=datasets, optimizer=optimizer, num_epochs=30)



created train
Epoch 1/30
----------
Train Loss: 0.1425 F1: 0.5134
Test Loss: 0.2059 F1: 0.5405

Epoch 2/30
----------
Train Loss: 0.1234 F1: 0.6201
Test Loss: 0.2007 F1: 0.5507

Epoch 3/30
----------
Train Loss: 0.1131 F1: 0.6449
Test Loss: 0.2046 F1: 0.5654

Epoch 4/30
----------
Train Loss: 0.1052 F1: 0.6760
Test Loss: 0.2172 F1: 0.5658

Epoch 5/30
----------
Train Loss: 0.0992 F1: 0.6970
Test Loss: 0.2163 F1: 0.5702

Epoch 6/30
----------
Train Loss: 0.0943 F1: 0.7169
Test Loss: 0.2247 F1: 0.5820

Epoch 7/30
----------
Train Loss: 0.0895 F1: 0.7339
Test Loss: 0.2138 F1: 0.6003

Epoch 8/30
----------
Train Loss: 0.0873 F1: 0.7427
Test Loss: 0.2331 F1: 0.5631

Epoch 9/30
----------
Train Loss: 0.0855 F1: 0.7459
Test Loss: 0.2220 F1: 0.6001

Epoch 10/30
----------
Train Loss: 0.0825 F1: 0.7531
Test Loss: 0.2486 F1: 0.5614

Epoch 11/30
----------
Train Loss: 0.0789 F1: 0.7561
Test Loss: 0.2472 F1: 0.5557

Epoch 12/30
----------
Train Loss: 0.0791 F1: 0.7655
Test Loss: 0.2561 F1: 0.5656


In [52]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.optim import Adam
from sklearn.metrics import f1_score

class SentimentLSTM(nn.Module):
    def __init__(self, embedding_dim, hidden_dim, num_classes, num_layers=1, dropout=0.5):
        super(SentimentLSTM, self).__init__()
        self.ignore_index = -100
        self.num_classes = num_classes
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        
        # LSTM layer
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, num_layers, batch_first=True, dropout=dropout)
        # Fully connected layer
        self.fc = nn.Linear(hidden_dim, num_classes)
        # Activation
        self.activation = nn.ReLU()
        # Dropout
        self.dropout = nn.Dropout(dropout)
        # Loss
        self.loss = nn.CrossEntropyLoss(ignore_index=-100)
        
    def forward(self, input_ids, labels=None):
        # LSTM output
        lstm_out, _ = self.lstm(input_ids)
        lstm_out = self.dropout(lstm_out)
        # Pass the output of the LSTM layer to the fully connected layer
        logits = self.fc(lstm_out)
        
        if labels is not None:
            # Flatten logits and labels to compute the loss
            logits_flat = logits.view(-1, logits.size(-1))
            labels_flat = labels.view(-1)
            loss = self.loss(logits_flat, labels_flat)
            return logits, loss
        
        return logits, None



In [54]:
# Assuming train_ds and test_ds are already defined and loaded
embedding_dim = 200  # Match this with your embedding dimension
hidden_dim = 100
num_classes = 2
num_layers = 1
dropout = 0.5

lstm_model = SentimentLSTM(embedding_dim, hidden_dim, num_classes, num_layers, dropout)
optimizer = Adam(lstm_model.parameters())

# Train the model
train(lstm_model, {"train": train_ds, "test": test_ds}, optimizer, num_epochs=30)


Epoch 1/30
----------
Train Loss: 0.1453 F1: 0.4781
Test Loss: 0.2020 F1: 0.5667

Epoch 2/30
----------
Train Loss: 0.1145 F1: 0.6241
Test Loss: 0.2022 F1: 0.5959

Epoch 3/30
----------
Train Loss: 0.1010 F1: 0.6818
Test Loss: 0.2040 F1: 0.5837

Epoch 4/30
----------
Train Loss: 0.0878 F1: 0.7214
Test Loss: 0.2065 F1: 0.6108

Epoch 5/30
----------
Train Loss: 0.0744 F1: 0.7755
Test Loss: 0.2137 F1: 0.6147

Epoch 6/30
----------
Train Loss: 0.0637 F1: 0.8084
Test Loss: 0.2351 F1: 0.5980

Epoch 7/30
----------
Train Loss: 0.0564 F1: 0.8377
Test Loss: 0.2453 F1: 0.6219

Epoch 8/30
----------
Train Loss: 0.0468 F1: 0.8703
Test Loss: 0.2593 F1: 0.6210

Epoch 9/30
----------
Train Loss: 0.0412 F1: 0.8843
Test Loss: 0.2771 F1: 0.6107

Epoch 10/30
----------
Train Loss: 0.0363 F1: 0.9027
Test Loss: 0.2925 F1: 0.6039

Epoch 11/30
----------
Train Loss: 0.0316 F1: 0.9157
Test Loss: 0.3281 F1: 0.6028

Epoch 12/30
----------
Train Loss: 0.0271 F1: 0.9280
Test Loss: 0.3731 F1: 0.5951

Epoch 13/30
-